In [ ]:
# Activate the environment that has qiime2 installed
# conda activate qiime2-amplicon-2024.10  

In [2]:
# %pip install biopython

In [ ]:
import os
import subprocess
import pandas as pd

Download the sra files

In [ ]:
# Download the accession IDs and load them into a list
accession_ids = []
with open("acc_list.txt", "r") as f:
    for line in f:
        accession_ids.append(line.strip())
sratoolkit_path = "sratoolkit.3.1.1-mac-arm64/bin" #path to the sratoolkit

for acc in accession_ids:
    # Use the actual path and accession ID
    result = subprocess.run([f"{sratoolkit_path}/prefetch", acc])
    print(result)

Split files into forward and backward reads

In [ ]:
filepath = "SRAS" #path to the folder where the SRAs are stored
for acc in accession_ids:
    # Use the actual path and accession ID
    result = subprocess.run([f"{sratoolkit_path}/fastq-dump", "--split-files", f"{filepath}/{acc}/{acc}.sra", "-O", f"FASTQ1/{acc}"])
    print(result)


Perform Quality Trimming

In [ ]:
files_dir = "FASTQ1"

for dirpath, dirnames, filenames in os.walk(files_dir):
    for filename in filenames:
        if filename.endswith("_1.fastq"):
            os.system("rm "+dirpath+"/"+filename.split('_')[0]+'_2.fastq')
            os.system("fastq_quality_trimmer -i "+dirpath+"/"+filename+" -o "+dirpath+"/"+filename.split('.')[0]+"_trimmed.fastq.gz -t 30 -l 80 -Q 33 -z")
            print("Done trimming "+filename)
    

Dereplicate the sequences and perform OTU picking for each sample

In [ ]:
files_dir = "FASTQ1"
# download reference gene from http://greengenes.microbio.me/greengenes_release/gg_13_8_otus/rep_set/97_otus.fasta

reference_file = "97_otus.fasta"
os.system("qiime tools import --type 'FeatureData[Sequence]' --input-path "+reference_file+" --output-path 97_otus.qza")
reference_sequences_path = "97_otus.qza"

for dirpath, dirnames, filenames in os.walk(files_dir):
    for filename in filenames:
        if filename.endswith("_trimmed.fastq.gz"):
            print(f"Processing {filename}")
            base_filename = filename.split('_')[0]
            filepath = dirpath

            # Rename the file to follow naming convention
            renamed_file = f"{base_filename}_L2S357_L001_R1_001.fastq.gz"
            os.system(f"mv {dirpath}/{filename} {dirpath}/{renamed_file}")

            # Remove any additional fastq file in the folder
            os.system(f"rm {dirpath}/{base_filename}_1.fastq")

            # Import the renamed FASTQ file to QIIME 2
            os.system(
                f"qiime tools import --type 'SampleData[SequencesWithQuality]' "
                f"--input-path {filepath} --input-format CasavaOneEightSingleLanePerSampleDirFmt "
                f"--output-path {filepath}/{base_filename}.qza"
            )

            # Dereplicate sequences
            os.system(
                f"qiime vsearch dereplicate-sequences --i-sequences {filepath}/{base_filename}.qza "
                f"--o-dereplicated-table {filepath}/{base_filename}_table.qza "
                f"--o-dereplicated-sequences {filepath}/{base_filename}_rep-seqs.qza"
            )

            # Closed-reference OTU picking
            os.system(
                f"qiime vsearch cluster-features-closed-reference "
                f"--i-table {filepath}/{base_filename}_table.qza "
                f"--i-sequences {filepath}/{base_filename}_rep-seqs.qza "
                f"--i-reference-sequences {reference_sequences_path} "
                f"--p-perc-identity 0.97 "
                f"--o-clustered-table {filepath}/{base_filename}_clustered-table.qza "
                f"--o-clustered-sequences {filepath}/{base_filename}_clustered-seqs.qza "
                f"--o-unmatched-sequences {filepath}/{base_filename}_unmatched-seqs.qza "
                f"--p-strand 'both'"
            )

            # Export the OTU table
            os.system(
                f"qiime tools export "
                f"--input-path {filepath}/{base_filename}_clustered-table.qza "
                f"--output-path {filepath}/{base_filename}_exported-feature-table"
            )

            # Convert the biom table to a tsv file
            os.system(
                f"biom convert -i {filepath}/{base_filename}_exported-feature-table/feature-table.biom "
                f"-o {filepath}/{base_filename}_exported-feature-table/feature-table.tsv "
                f"--to-tsv"
            )

            print(f"Completed processing for {filename}")

Merge the individual OTU tables into one table for the entire dataset

In [ ]:
files_dir = "FASTQ1"

merged_df = pd.DataFrame()

for sample_dir in os.listdir(files_dir):
    sample_path = os.path.join(files_dir, sample_dir)
    
    if os.path.isdir(sample_path):
        for nested_dir in os.listdir(sample_path):
            nested_path = os.path.join(sample_path, nested_dir)
            
            if os.path.isdir(nested_path):
                for filename in os.listdir(nested_path):
                    if filename.endswith("feature-table.tsv"):
                        tsv_path = os.path.join(nested_path, filename)
                        
                        sample_id = sample_dir

                        df = pd.read_csv(tsv_path, sep="\t", skiprows=2)
                        
                        if df.columns[0] != "#OTU ID":
                            df.rename(columns={df.columns[0]: "#OTU ID"}, inplace=True)
                        
                        if len(df.columns) > 1:
                            df = df.rename(columns={df.columns[1]: sample_id})
                        
                        if merged_df.empty:
                            merged_df = df 
                        else:
                            merged_df = pd.merge(merged_df, df, on="#OTU ID", how="outer")


merged_df.fillna(0, inplace=True)

merged_df.to_csv("merged_feature_table.tsv", sep="\t", index=False)
print("Merged table saved as merged_feature_table.tsv")


Normalize the merged OTU table and add the sample labels from metadata

In [ ]:
merged_df = pd.read_csv("merged_feature_table.tsv", sep="\t")
normalized_df = merged_df.set_index("#OTU ID").T

normalized_df = normalized_df.div(normalized_df.sum(axis=1), axis=0) * 1000

metadata = pd.read_csv("metadata.csv")
labels = {}
for index, row in metadata.iterrows():
    if row['sample_id'].startswith('C'):
        labels[row['Run']] = "control"
    elif row['sample_id'].startswith('M'):
        labels[row['Run']] = "mci"
    else:
        labels[row['Run']] = "alzheimer"

normalized_df['class'] = normalized_df.index.map(labels)
normalized_df = normalized_df.reset_index(drop=True)
print(normalized_df.columns)
normalized_df.to_csv("normalized_feature_table_three_class.csv", index=False)


Check the labels and make sure the data was processed correctly

In [ ]:
# load the normalized table and print the distribution of the labels
normalized_df = pd.read_csv("normalized_feature_table.csv")
print(normalized_df['class'].value_counts())

print(normalized_df.shape[0])